In [1]:

import argparse

from loaders import CxrDataLoader
from torch.utils.data import DataLoader
from pretrained.blip_finetune import BLIPTrainer
from pretrained.blip_datasets import BlipMimicCXRDataset
from transformers import BlipProcessor, BlipForConditionalGeneration

/Users/tudormihaita/PycharmProjects/clip-cxr-report-gen/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
args = argparse.Namespace(
    dataset_name='mimic-cxr',
    batch_size=64,
    num_workers=0,
    max_seq_length=512,
    use_minio=False
)

In [2]:
# from PIL import Image
# from transformers import AutoProcessor, AutoModelForImageTextToText
#
# processor = AutoProcessor.from_pretrained("adibvafa/BLIP-MIMIC-CXR")
# model = AutoModelForImageTextToText.from_pretrained("adibvafa/BLIP-MIMIC-CXR")

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like adibvafa/BLIP-MIMIC-CXR is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [3]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

In [6]:
data_loader = CxrDataLoader(args, split='test', transform=None)
sample = next(iter(data_loader))

image = sample['image'][0]
image = image.unsqueeze(0)
report = sample['report'][0]

inputs = processor(images=image, return_tensors="pt")

outputs = model.generate(**inputs)
print(processor.decode(outputs[0], skip_special_tokens=True))

there is a man that is standing in the dark with a cell phone


In [4]:
train_dataset = BlipMimicCXRDataset(args, 'train', processor)
val_dataset = BlipMimicCXRDataset(args, 'val', processor)
test_dataset = BlipMimicCXRDataset(args, 'test', processor)

In [5]:
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers),
    'val': DataLoader(val_dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers),
    'test': DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False, num_workers=args.num_workers)
}

In [6]:
trainer = BLIPTrainer(
    model,
    processor,
    train_loader=dataloaders['train'],
    val_loader=dataloaders['val'],
    test_loader=dataloaders['test'],
    mixed_precision=False,
    log_interval=50,
    max_epochs=1
)

In [ ]:
trainer.train()

2025-04-14 13:31:19,595 - INFO - Starting training
2025-04-14 13:31:19,596 - INFO - Epoch 1
Training:   0%|          | 0/3427 [00:00<?, ?it/s]